<a href="https://colab.research.google.com/github/Theallkeeeymist/duplicate-question-pair-nlp/blob/main/duplicate_question_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("/content/train.csv")
df.shape

(404290, 6)

In [3]:
df=df.sample(90000)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
290486,290486,60721,116540,Why aren't we talking about animal farming's i...,Why don't environmental organisations in the U...,1
154758,154758,242604,242605,What were Machiavelli's political views?,"Were you or anyone you know a Tupamaro? If so,...",0
204327,204327,307191,307192,How can I have sex with my teacher?,How do I make my teacher have a crush on me?,0
378535,378535,509929,509930,When did the laws change to if you're convicte...,How can I get information about a service prov...,0
169464,169464,262244,262245,What is the easiest way to remove stains of Fe...,What is the easiest way to remove stains of Fe...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90000 entries, 290486 to 60165
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            90000 non-null  int64 
 1   qid1          90000 non-null  int64 
 2   qid2          90000 non-null  int64 
 3   question1     90000 non-null  object
 4   question2     90000 non-null  object
 5   is_duplicate  90000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.8+ MB


In [6]:
df['question1']=[data if isinstance(data,str) else "" for data in df["question1"]]
df['question2']=[data if isinstance(data,str) else "" for data in df["question2"]]

In [7]:
df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
# Distribution of dataset

print(df["is_duplicate"].value_counts())

is_duplicate
0    56745
1    33255
Name: count, dtype: int64


In [10]:
# Distribution in percentage

print((df["is_duplicate"].value_counts()/df["is_duplicate"].count())*100)

is_duplicate
0    63.05
1    36.95
Name: count, dtype: float64


In [11]:
# Repeated questions

qid=pd.Series(df['qid1'].tolist()+df['qid2'].tolist())
print(f"Number of unique questions: {np.unique(qid).shape[0]}")

Number of unique questions: 151272


In [12]:
ques_df=df[['question1','question2']]
ques_df.head()

,question1,question2
290486,Why aren't we talking about animal farming's i...,Why don't environmental organisations in the U...
154758,What were Machiavelli's political views?,"Were you or anyone you know a Tupamaro? If so,..."
204327,How can I have sex with my teacher?,How do I make my teacher have a crush on me?
378535,When did the laws change to if you're convicte...,How can I get information about a service prov...
169464,What is the easiest way to remove stains of Fe...,What is the easiest way to remove stains of Fe...


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

#text merging
questions=list(ques_df['question1'])+list(ques_df['question2'])

cv=CountVectorizer(max_features=2000)
q1_arr,q2_arr=np.vsplit(cv.fit_transform(questions).toarray(),2)

In [14]:
temp_df1=pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2=pd.DataFrame(q2_arr, index=ques_df.index)
temp_df=pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(90000, 4000)

In [15]:
temp_df['is_duplicate']=df['is_duplicate']

In [16]:
temp_df.shape

(90000, 4001)

In [17]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state=42)

In [20]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model=XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)

0.7343333333333333